In [19]:
import pandas as pd
import numpy as np
from scapy.all import *

def ZeekLogs_to_csv(file_path):
    try:
        out_data = pd.read_excel(file_path + '.xlsx')
        # out_data = pd.read_csv(file_path + '.csv')
        return out_data
    except:
        data_file = open(file_path)
        line = data_file.readline()
        attribs = {}
        while line.strip().startswith('#'):
            # print(line)
            key, *val = line.split()
            attribs[key[1:]] = val
            line = data_file.readline()
        # print(attribs)
        df = {}
        while line.strip().startswith('#close') is False:
            for k, v in zip(attribs['fields'], line.split()):
                # print(k, v)
                if k not in df.keys():
                    df[k] = []
                df[k].append(v)
            line = data_file.readline()

        data_file.close()
        out_data = pd.DataFrame(df)
        out_data.to_csv(file_path + '.csv', index=False)
        return out_data
    
def preprocess_data(file):
    data = ZeekLogs_to_csv(file)
    data['detailed-label'] = data['detailed-label'].replace(to_replace= '-',value= 'Benign')
    data = data.replace(to_replace='-', value=pd.NA)
    data = data.fillna(value=0)
    data['ts'] = data['ts'].astype(np.float64)
    return data

def csv_to_df(csv_file):
    print("[*] Loading:{0}".format(csv_file), end = "\r")
    data = preprocess_data(csv_file)
    df = data[['ts','detailed-label']]
    print("[+] Load Completed:{0}".format(csv_file))
    return df

def pcap_to_df(pcap_file, byline = True):
    print("[*] Loading:{0}".format(pcap_file), end = "\r")    
    if byline == True:
        packets = PcapReader(pcap_file)
        length = "Unknown"
    else:
        packets = rdpcap(pcap_file)
        length = len(packets)
    print("[+] Load Completed:{0}".format(pcap_file))
    df = pd.DataFrame({"ts": []})
    i = 0
    for data in packets:
        print("[*] Prasing Progress:{0}/{1}".format(i,length), end = "\r")
        df.loc[i] = data.time
        i += 1

    print("[+] Prased Items:{0}              ".format(i))
    print("[+] Prase Completed:{0}".format(pcap_file))
    return df.astype(np.float64)

def find_pcap(folder):
    for file in os.listdir(folder):
        if os.path.splitext(file)[1] == '.pcap':
            if str.find(file, '2018') != -1 or str.find(file, '2019') != -1:
                return os.path.join(folder, file)
    

In [77]:
location = '/Volumes/Unicorn/IoTScenarios/'
folder_list = ['CTU-IoT-Malware-Capture-42-1/']
folder = location + folder_list[0]
zeek_df = csv_to_df(folder + 'bro/conn.log.labeled')

[+] Load Completed:/Volumes/Unicorn/IoTScenarios/CTU-IoT-Malware-Capture-42-1/bro/conn.log.labeled


In [78]:
zeek_df

,ts,detailed-label
0,1.547127e+09,C&C-FileDownload
1,1.547127e+09,Benign
2,1.547127e+09,Benign
3,1.547127e+09,Benign
4,1.547127e+09,C&C-FileDownload
...,...,...
4420,1.547156e+09,Benign
4421,1.547156e+09,Benign
4422,1.547157e+09,Benign
4423,1.547127e+09,Benign


In [71]:
pcap_df = pcap_to_df(find_pcap(folder), byline=True)
pcap_df

[+] Load Completed:/Volumes/Unicorn/IoTScenarios/CTU-IoT-Malware-Capture-42-1/2019-01-10-14-34-38-192.168.1.197.pcap
[+] Prased Items:24485              
[+] Prase Completed:/Volumes/Unicorn/IoTScenarios/CTU-IoT-Malware-Capture-42-1/2019-01-10-14-34-38-192.168.1.197.pcap


,ts
0,1.547127e+09
1,1.547127e+09
2,1.547127e+09
3,1.547127e+09
4,1.547127e+09
...,...
24480,1.547157e+09
24481,1.547157e+09
24482,1.547157e+09
24483,1.547157e+09


In [72]:
pcap_df['ts'] = (pcap_df['ts'] * 100000)//1/100000
pcap_df['ts'][43]

1547127288.01882

In [63]:
# from math import floor
# floor()

df = pd.DataFrame([[1547127288.01882, 1], [1547127288.018828, 4]], columns=['uid', 'score'])
df['uid'] = round(df['uid'])


a = 

a = 1547127288.018828
a = 1547127288.018811
b = a - 0.000005
print(b)
b = round(b, 5)
print(b)

assert(a == b)

1547127288.018806
1547127288.01881


AssertionError: 

In [74]:
result = pd.merge(pcap_df, zeek_df, how='left', left_on=['ts'], right_on='ts')
result = result[['detailed-label']]
result

,detailed-label
0,NaN
1,NaN
2,Benign
3,NaN
4,NaN
...,...
24484,NaN
24485,NaN
24486,NaN
24487,NaN


In [75]:
result['detailed-label'].value_counts()

Benign              4433
C&C-FileDownload       4
FileDownload           2
Name: detailed-label, dtype: int64

In [76]:
result.to_csv(folder + 'pcap_label.csv')
print('[+] File Saved:'+ folder + 'pcap_label.csv')

[+] File Saved:/Volumes/Unicorn/IoTScenarios/CTU-IoT-Malware-Capture-42-1/pcap_label.csv


In [79]:
a = pd.read_excel(folder + 'bro/conn.log.labeled.xlsx')
a.to_csv(folder + 'bro/conn.log.labeled.csv')